In [ ]:
#if (!require("BiocManager", quietly = TRUE))
  #install.packages("BiocManager")
#BiocManager::install("DOSE")
library(clusterProfiler)
library(tidyverse)
library(forcats)
library(ggrepel)
library(enrichplot)
library(tibble)
library(ggupset)
library(ggridges)
library(gage)
library(DOSE)
library(pathview)
library(ggpubr)

# Generate ranked list of ko_id's and LFC
numeric vector: fold change or other type of numerical variable
- here I sorted by differential expression, taking the sign from LFC and value of significance from adjusted pvalue.
- top: most upregulated genes
- middle: most insignificantly expressed genes
- bottom: most downregulated genes

named vector: every number has a name, the corresponding gene ID
sorted vector: number should be sorted in decreasing order
import your data from a csv file, the file should contains two columns, one for gene ID (no duplicated ID allowed) and another one for fold change.
### Over expression analysis uses only DE genes
### Gene set expression analysis uses all genes with DE values

In [ ]:
read.data = function(df){
    df = read.csv(paste("./de_res_files/",df,".csv",sep=''))
    colnames(df)[1] = 'ko_id'
    df = filter(df, padj<=0.05)
    # create a ranked gene list
    genes=df$log2FoldChange
    names(genes)=as.character(df$ko_id)
    geneList=sort(genes, decreasing=T)
    # geneList is a sorted list of genes with thier DE ranking and ko_id as list names
    print(max(geneList))
    geneList
    }

                       

In [ ]:
#High vs Low Fe
HvL8.geneList = read.data('lfc8.k.HvL')
HvL6.geneList = read.data('lfc6.k.HvL')
HvL13.geneList = read.data('lfc13.k.HvL')
#Fe ammendment vs Low Fe
AvL4.geneList = read.data('lfc4.k.AvL')
AvL8.geneList = read.data('lfc8.k.AvL')
AvL6.geneList = read.data('lfc6.k.AvL')
AvL13.geneList = read.data('lfc13.k.AvL')
#Fe ammendment vs High Fe
AvH8.geneList = read.data('lfc8.k.AvH')
AvH6.geneList = read.data('lfc6.k.AvH')
AvH13.geneList = read.data('lfc13.k.AvH')
tail(AvH13.geneList)

# Pull our Kegg Metabolism and cell cycle pathways
This will only compare ko's to those which are annotated to metabolism and cell cycle pathways and not human diseases. 

In [ ]:
kg.ko=kegg.gsets(species='ko', check.new=T) 
kegg.sigmet=kg.ko$kg.sets[kg.ko$sigmet.idx]
kegg.sigmet.term2gene = data.frame(
    TERM=str_remove(rep(names(kegg.sigmet), sapply(kegg.sigmet, length)), 'ko[[:digit:]]{5}'), 
    GENE= unlist(kegg.sigmet))

kegg.met = kg.ko$kg.sets[kg.ko$met.idx]
kegg.gs.term2gene = data.frame(
    TERM=str_remove(rep(names(kegg.met), sapply(kegg.met, length)), 'ko[[:digit:]]{5}'), 
    GENE= unlist(kegg.met))
head(kegg.gs.term2gene)

In [ ]:
a=kegg.met[1]
a=data.frame(ko_id=unlist(a))
#a= a %>% rownames_to_column('Path_name') 
head(a)
#a$pat=str_split(a$Path_name, 'ko[[:digit:]]*', simplify = T)
b = str_split_fixed(rownames(a), ' ',n = 2)
b = as_tibble(b, .name_repair = make.names)
colnames(b) = c('path_code','path_name')
c=bind_cols(ko_id=a$ko_id, b)
head(c)


# Run GSE analysis 

In [ ]:
run.gsea = function(geneList){
    gsea = GSEA(geneList, minGSSize=5, eps = 0,
                pvalueCutoff = 0.05, 
                pAdjustMethod = 'fdr',
                TERM2GENE = kegg.gs.term2gene)
    gsea    
}              

In [ ]:
gsea.HvL8=run.gsea(HvL8.geneList)
gsea.AvL8=run.gsea(AvL8.geneList)
gsea.AvH8=run.gsea(AvH8.geneList)

dotplot(gsea.HvL8, showCategory=100,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvL8, showCategory=100,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvH8, showCategory=100,split=".sign") + 
                    facet_grid(.~.sign)

In [ ]:
gsea.HvL6=run.gsea(HvL6.geneList)
gsea.AvL6=run.gsea(AvL6.geneList)
gsea.AvH6=run.gsea(AvH6.geneList)

dotplot(gsea.HvL6, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvL6, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvH6, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)


In [ ]:

gsea.AvL4=run.gsea(AvL4.geneList)
dotplot(gsea.AvL4, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)


In [ ]:
gsea.HvL13=run.gsea(HvL13.geneList)
gsea.AvL13=run.gsea(AvL13.geneList)
gsea.AvH13=run.gsea(AvH13.geneList)

dotplot(gsea.HvL13, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvL13, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)
dotplot(gsea.AvH13, showCategory=30,split=".sign") + 
                    facet_grid(.~.sign)

## Pull out result DF and add organism name
Create a df for each comparison 

In [ ]:
Avl8=mutate(gsea.AvL8@result, organism='C. closterium UGA8')
Avl4=mutate(gsea.AvL4@result, organism='C. closterium UGA4')
Avl6=mutate(gsea.AvL6@result, organism='G. oceanica')
Avl13=mutate(gsea.AvL13@result, organism='G. huxleyi')

Avh8=mutate(gsea.AvH8@result, organism='C. closterium UGA8')
Avh6=mutate(gsea.AvH6@result, organism='G. oceanica')
Avh13=mutate(gsea.AvH13@result, organism='G. huxleyi')

Hvl8=mutate(gsea.HvL8@result, organism='C. closterium UGA8')
Hvl6=mutate(gsea.HvL6@result, organism='G. oceanica')
Hvl13=mutate(gsea.HvL13@result, organism='G. huxleyi')

Hvl = bind_rows(Hvl13,Hvl8,Hvl6)
Avl = bind_rows(Avl4, Avl13, Avl8, Avl6)
Avh = bind_rows(Avh13, Avh8, Avh6)


## Plot enrichments

## Make dot plots
First, make treatment column and filter enrichment by positive/negative NES
Make dotplot with organisms on X and Description on Y. 

In [ ]:
Hvl$treatment = ifelse(Hvl$NES > 0, 'High Iron','Low Iron')
Avl$treatment = ifelse(Avl$NES > 0, 'Iron Amendment', 'Low Iron')
Avh$treatment = ifelse(Avh$NES > 0, 'Iron Amendment', 'High Iron')

all.gsea.dotplot = function(df){
    ggplot(df, aes(organism, Description, color=p.adjust, size=abs(NES)))+ 
   geom_point(stat = 'identity', size=20)+
facet_grid(~treatment, scales='free_y')+
theme_bw()+
xlab(NULL) + 
ylab(element_blank()) +
scale_y_discrete(labels=function(x) str_wrap(x, 35))+ #, expand=c(0.02,0.02)) +
scale_x_discrete(labels=function(x) str_wrap(x, 15)) +
scale_color_gradient(low = 'red', high='blue')+
guides(color=guide_colorbar(title = 'Normalized enrichment score', reverse=TRUE, 
                            title.theme = element_text(size=20, face='bold'),
                           title.position='top', label.theme=element_text(size=20),
                           barwidth=30, barheight=2)) +
theme(text = element_text(color='black', size=20), 
          panel.grid.minor = element_line(colour = 'grey'), 
          strip.text=element_text(size=25, face='bold'), 
          strip.background=element_rect(fill='white'), 
          axis.text.y=element_text(size=20, color='black'), 
          axis.text.x = element_text(size=20, face = 'italic', color='black'),
    plot.margin=margin(0.2,0.2,0.2,3, unit='cm'))
}

#png(file = '../figs/gsea_dotplot_HvL.png',width = 10, height = 15)
hvl.gsea.dotplot = all.gsea.dotplot(Hvl)    
#print(hvl.gsea.dotplot)
#dev.off()
#ggsave(filename = '../figs/gsea_dotplot_HvL.png', hvl.gsea.dotplot, device='png', height = 10, width=15)
                 
#png(file = '../figs/gsea_dotplot_AvL.png',width = 10, height = 15)
avl.gsea.dotplot = all.gsea.dotplot(Avl) 
#print(avl.gsea.dotplot)
#dev.off()
#ggsave(filename = '../figs/gsea_dotplot_AvL.png', avl.gsea.dotplot, device='png', height = 5, width=8)
                 
#png(file = '../figs/gsea_dotplot_AvH.png',width = 10, height = 15)
avh.gsea.dotplot = all.gsea.dotplot(Avh)                
#print(avh.gsea.dotplot)
#dev.off()
# ggsave(filename = '../figs/gsea_dotplot_AvH.png', avh.gsea.dotplot, device='png', height = 15, width=15)
                 
avl.gsea.dotplot

In [ ]:
                                  
#gsea_dotplot_all= ggarrange(avl.gsea.dotplot, hvl.gsea.dotplot, ncol=1, common.legend=T, align='v', legend='right')      
                 
#ggsave(filename = '../figs/gsea_dotplot_all.png', gsea_dotplot_all, device='png', height = 25, width=19)

gsea_dotplot_all= ggarrange(avl.gsea.dotplot, hvl.gsea.dotplot, ncol=1, common.legend=T, font.label = element_text(size=30, face='bold'),
                            align='v', legend='bottom',labels = c('A','B'))  
                 
ggsave(filename = '../figs/gsea_dotplot_all2.png', gsea_dotplot_all, device='png', height = 25, width=23)

In [ ]:
gsea_dotplot_all

# Make table of enrichment and genes within pathways.

In [ ]:
colnames(Hvl)
split_into_multiple <- function(column, pattern = "/", into_prefix){
    #adapted from post on stack overflow
    cols <- str_split_fixed(column, pattern, n = Inf)
  # Replace empty matrix indicies with NA's 
    cols[which(cols == "")] <- NA
  # turn matrix into a table with unique but arbitraty column names  
    cols <- as_tibble(cols, .name_repair = make.names)
  # where m = # columns in tibble 'cols'
    m <- dim(cols)[2]
  # assign column names as 'into_prefix_1', 'into_prefix_2', ..., 'into_prefix_m' 
    names(cols) <- paste(into_prefix, 1:m, sep = "_")
    print('# of values in matrix w/o NA:')
    print(sum(!is.na(cols)))
    return(cols)
}

clean_ko <- function(df){
    ko_iterations <- split_into_multiple(df$core_enrichment, "/", "ko:")
    df = df %>% select(c('Description','organism','treatment','NES')) %>% bind_cols(ko_iterations)
    df_clean = pivot_longer(df, cols = !c('Description','organism','treatment','NES'), 
                            values_drop_na = T,
                            values_to = 'ko_id', names_to = 'ko_count')
    df_clean=df_clean %>% select(!'ko_count')
    df_clean
    }

core.ko.Hvl=clean_ko(Hvl)
core.ko.Avl=clean_ko(Avl)
core.ko.Avh=clean_ko(Avh)
head(core.ko.Avl)

In [ ]:
ko.def=read.csv('../kegg_names/ko_def.csv')
ko.def$name=str_remove(ko.def$name, '\\[EC:.+')

core.ko.Hvl = right_join(ko.def, core.ko.Hvl, by='ko_id')
core.ko.Avl = right_join(ko.def, core.ko.Avl, by='ko_id')
core.ko.Avh = right_join(ko.def, core.ko.Avh, by='ko_id')


In [ ]:
write.csv(core.ko.Hvl,'./de_res_files/core.ko.Hvl.csv',  row.names=F)
write.csv(core.ko.Avl,'./de_res_files/core.ko.Avl.csv',  row.names=F)
write.csv(core.ko.Avh,'./de_res_files/core.ko.Avh.csv',  row.names=F)